# Preliminary feature gen and splitting

In [1]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score, confusion_matrix
import random
import torch
#import nltk
from torch import nn, optim
from collections import OrderedDict

In [2]:
def isFirstCap(x):
    if x[0].isupper():
        return 1
    else:
        return 0

def Length(x):
    return len(x)

def endY(x):
    if x[-1] == 'y':
        return 1
    else:
        return 0

def endan(x):
    if x[-2:len(x)] == 'an':
        return 1
    else:
        return 0

def isNum(x):
    if x.isnumeric():
        return 1
    else:
        return 0

def endS(x):
    if x[-1] == 's':
        return 1
    else:
        return 0
    
def endish(x):
    if x[-3:len(x)] == 'ish':
        return 1
    else:
        return 0

def endese(x):
    if x[-3:len(x)] == 'ese':
        return 1
    else:
        return 0

def otherCap(x):
    for letter in x:
        if letter.isupper():
            return 1
    return 0

def propVow(x):
    vowels = 'aeiouAEIOU'
    numVow = 0
    for letter in x:
        if letter in vowels:
            numVow += 1
    return numVow / len(x)

def frontWord(x, array, df):
    if x > 0:
        return array.index(df['Word'][x-1])
    else:
        return -1
    
def backWord(x, array, df):
    if x < len(df.index) - 1:
        return array.index(df['Word'][x+1])
    else:
        return -2

def Array2Num(x, array):
    return array.index(x)

In [3]:
filename = 'Dataset_08-29-2019.txt'
df = pd.read_csv(filename, sep = '\t', encoding = 'unicode_escape')
word_data = df['Word'].values.tolist()
#word_vec = [nltk.word_tokenize(title) for title in word_data]
size = len(word_data) 
idx_list = [idx + 1 for idx, val in
            enumerate(word_data) if val == '.'] 
res = [word_data[i: j] for i, j in
        zip([0] + idx_list, idx_list + 
        ([size] if idx_list[-1] != size else []))] 
model = Word2Vec(res, size=24, window=5, min_count=0, workers=4)
model.save('word2vec.model')

In [4]:
function_dict = {'isFirstCap': isFirstCap, 'Length': Length, 'endY': endY, 'otherCap': otherCap, 'endan': endan,
                  'isNum': isNum, 'endS': endS, 'endish': endish, 'endese': endese, 'propVow': propVow}
for f in function_dict:
    df[f] = df['Word'].apply(lambda x: function_dict[f](x))
wv = KeyedVectors.load('word2vec.model')



#NEED TO FIX: WORD2VEC DOESN'T WORK WHEN SAVING TO CSV AND READING IT AGAIN



for i in range(24):
    df['WordVector' + str(i)] = df['Word'].apply(lambda x: wv[x][i] if x in wv else 0)
#df['WordVector'] = df['Word'].apply(lambda x: wv[x] if x in wv else None)
tag_array = df.Tag.unique().tolist()
df['TagNum'] = df['Tag'].apply(lambda x: Array2Num(x, tag_array))
POS_array = df.POS.unique().tolist()
df['POSNum'] = df['POS'].apply(lambda x: Array2Num(x, POS_array))
word_array = df.Word.unique().tolist()
df['frontWord'] = df['Unnamed: 0'].apply(lambda x: frontWord(x, word_array, df))
df['backWord'] = df['Unnamed: 0'].apply(lambda x: backWord(x, word_array, df))

C:\Users\jaken\anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
C:\Users\jaken\anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [5]:
sentences_group = df.groupby(['Sentence #'])
test_sentences = []
test_dfs = []
train_dfs = []
max_sent = int(df['Sentence #'].max())
for i in range(round(max_sent / 8)):
    found = False
    while found == False:
        num = random.randint(1, max_sent)
        if not num in test_sentences:
            test_sentences.append(num)
            test_dfs.append(sentences_group.get_group(num))
            found = True
for i in range(round(max_sent / 8)):
    found = False
    while found == False:
        num = random.randint(1, max_sent)
        if not num in test_sentences:
            test_sentences.append(num)
            test_dfs.append(sentences_group.get_group(num))
            found = True
            
test_df = pd.concat(test_dfs)
drop_list = test_df['Unnamed: 0'].tolist()
train_df = df.copy().drop(drop_list)
feature_list = ['isFirstCap', 'Length', 'endY', 'otherCap', 'endan',
                   'isNum', 'endS', 'endish', 'endese', 'propVow', 'POSNum', 'frontWord', 'backWord']
vectorlist = []
for i in range(24):
    vectorlist.append('WordVector' + str(i))
feature_list = feature_list + vectorlist
data_train = train_df[feature_list].values
label_train = train_df['TagNum'].values
data_test = test_df[feature_list].values
label_test = test_df['TagNum'].values

In [19]:
sentences_group.get_group(2)

,Unnamed: 0,Sentence #,Word,POS,Tag,isFirstCap,Length,endY,otherCap,endan,...,WordVector18,WordVector19,WordVector20,WordVector21,WordVector22,WordVector23,TagNum,POSNum,frontWord,backWord
24,24,2.0,Families,NNS,O,1,8,0,1,0,...,0.010567,0.012247,-0.003722,0.010851,0.004688,0.011165,0,0,21,1
25,25,2.0,of,IN,O,0,2,0,0,0,...,-1.443563,0.424729,-0.311578,1.228195,4.411615,1.918454,0,1,22,23
26,26,2.0,soldiers,NNS,O,0,8,0,0,0,...,-0.302861,0.094733,-0.069573,0.299913,1.010724,0.433820,0,0,1,24
27,27,2.0,killed,VBN,O,0,6,0,0,0,...,-0.744413,0.196432,-0.219188,0.726316,2.496732,1.094427,0,3,23,11
28,28,2.0,in,IN,O,0,2,0,0,0,...,-1.355045,0.427268,-0.343154,1.210463,4.213583,1.856411,0,1,24,9
29,29,2.0,the,DT,O,0,3,0,0,0,...,-1.395183,0.381858,-0.342208,1.179778,4.213542,1.832739,0,7,11,25
30,30,2.0,conflict,NN,O,0,8,0,0,0,...,-0.084495,0.041270,-0.003541,0.085129,0.314747,0.138048,0,8,9,26
31,31,2.0,joined,VBD,O,0,6,0,0,0,...,-0.093136,0.009367,-0.014573,0.087761,0.240624,0.117359,0,12,25,9
32,32,2.0,the,DT,O,0,3,0,0,0,...,-1.395183,0.381858,-0.342208,1.179778,4.213542,1.832739,0,7,26,27
33,33,2.0,protesters,NNS,O,0,10,0,0,0,...,-0.179721,0.050877,-0.052295,0.173662,0.572546,0.251046,0,0,9,28


# PyTorch NN

In [20]:
#len(feature_list)
input_size = 37
hidden_sizes = [30, 23]
#length of num_classes
output_size = 17

model = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(input_size, hidden_sizes[0])),
    ('relu1', nn.ReLU()),
    ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
    ('relu2', nn.ReLU()),
    ('output', nn.Linear(hidden_sizes[1], output_size)),
    ('softmax', nn.Softmax(dim=1))
]))

'''
multiplier_list = []
for num in range(output_size):
    multiplier_list.append(np.count_nonzero(label_train == num))
'''
weights = torch.tensor([1,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5])

criterion = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(model.parameters(), lr=0.00003)

In [25]:
epochs = 10

x_factor = np.count_nonzero(label_train == 0)
for i in range(epochs):
    running_loss = 0.0
    for idx, word in enumerate(data_train):
        label = label_train[idx]
        optimizer.zero_grad()
        word = np.array([word])
        word = torch.from_numpy(word)
        output = model(word.float())
        target_list = []
        for i in range(output_size):
            if i == label:
                if i == 0:
                    target_list.append(float(1))
                else:
                    target_list.append(float(3))
            else:
                target_list.append(float(0))
        target = torch.tensor([target_list])
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(data_train)}")
        
correct = 0
total = 0
total_list = []
for idx, guess in enumerate(data_test):
    word = np.array([guess])
    word = torch.from_numpy(word)
    output = model(word.float())
    guess_list = output[0].detach().numpy().tolist()
    total_list.append(guess_list)


classed_data = [np.argmax(line) for line in total_list]
accuracy = accuracy_score(classed_data, label_test)
cm = confusion_matrix(label_test, classed_data)
print('Accuracy Score: ' + str(accuracy))
print(cm)

Training loss: 1.30704628084407
Training loss: 1.3034968411039916
Training loss: 1.2936676207244115
Training loss: 1.2925609007643075
Training loss: 1.2936651453426053
Training loss: 1.292360126636491
Training loss: 1.2879250919927714
Training loss: 1.284025960593684
Training loss: 1.2837773796578509
Training loss: 1.2855848310766373
Accuracy Score: 0.8806800992315604
[[13878    66    87     0     0    47     0    24     0     0     0     0
      0     0     0     0     0]
 [   35   397     5     0     0    70     0     4     0     0     0     0
      0     0     0     0     0]
 [   46    51   148     0     0    28     0     2     0     0     0     0
      0     0     0     0     0]
 [   65    60     6     0     0   136     0     3     0     0     0     0
      0     0     0     0     0]
 [   46    13     3     0     0    15     0     1     0     0     0     0
      0     0     0     0     0]
 [   39   156     8     0     0    98     0     5     0     0     0     0
      0     0     0 

In [40]:

    
    
    
#TO FIX!!!! MODEL ONLY KICKS OUT 0'S





# lightgbm and normal project stuff

In [29]:
d_train = lgb.Dataset(data_train, label=label_train)
params = {}
params['objective'] = 'multiclass'
params['num_classes'] = 17
params['learning_rate'] = 0.03
params['colsample_bytree'] = 0.7
params['max_depth'] = 20
params['min_split_gain'] = 0.3
params['n_estimators'] = 400
params['num_leaves'] = 100
params['reg_alpha'] = 1.1
params['reg_lambda'] = 1.3
params['subsample'] = 0.9
params['subsample_freq'] = 20
mod = lgb.train(params, d_train, 100)
prediction_data = mod.predict(data_test)
classed_data = [np.argmax(line) for line in prediction_data]
accuracy = accuracy_score(classed_data, label_test)
cm = confusion_matrix(label_test, classed_data)
print(accuracy)
print(cm)

0.9543776850003025
[[14042    10     5    18     1     3     6     4     0     0     9     0
      4     0     0     0     0]
 [   13   429    17     6     3    34     2     2     0     0     4     0
      0     0     1     0     0]
 [    9    36   215     2     1     7     3     0     0     0     2     0
      0     0     0     0     0]
 [    6    15     2   205     0    11     4     1     0     0    26     0
      0     0     0     0     0]
 [    6     7     1     1    42     2    11     0     0     0     8     0
      0     0     0     0     0]
 [   19    56     9    13     1   182    13     1     0     0    10     0
      0     0     2     0     0]
 [   35     8     0    14     6     8   125     0     0     1    28     0
      0     0     0     1     0]
 [   54     4     0     0     0     1     0   226     0     0     1     0
      5     0     0     0     0]
 [    2     4     0     2     0     5     0     0     1     0     1     0
      0     0     0     0     0]
 [    0     0     